In [1]:
pip install opencv-python mediapipe pyautogui

  Using cached opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached mediapipe-0.10.14-cp311-cp311-win_amd64.whl.metadata (9.9 kB)
  Using cached PyAutoGUI-0.9.54.tar.gz (61 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached jax-0.4.28-py3-none-any.whl.metadata (23 kB)
  Using cached jaxlib-0.4.28-cp311-cp311-win_amd64.whl.metadata (1.8 kB)
  Using cached opencv_contrib_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached protobuf-4.25.3-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached sounddevice-0.

In [ ]:
import cv2
import mediapipe as mp
import pyautogui

# Initialize video capture
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Initialize Mediapipe hand detector and drawing utilities
hand_detector = mp.solutions.hands.Hands()
drawing_utils = mp.solutions.drawing_utils

# Get screen dimensions
screen_width, screen_height = pyautogui.size()

try:
    while True:
        # Read a frame from the webcam
        ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

        # Flip and convert the frame to RGB
        frame = cv2.flip(frame, 1)
        frame_height, frame_width, _ = frame.shape
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame to detect hands
        output = hand_detector.process(rgb_frame)
        hands = output.multi_hand_landmarks

        # If hands are detected
    if hands:
        for hand in hands:
            # Draw landmarks on the frame
            drawing_utils.draw_landmarks(frame, hand)
            landmarks = hand.landmark
            # Initialize coordinates for the index finger and thumb
             index_x, index_y = 0, 0
            thumb_x, thumb_y = 0, 0

                for id, landmark in enumerate(landmarks):
                    x = int(landmark.x * frame_width)
                    y = int(landmark.y * frame_height)

                    # Index finger (landmark 8)
                    if id ==  8:
                        cv2.circle(frame, (x, y), 10, (0, 255, 255), -1)
                        index_x = screen_width / frame_width * x
                        index_y = screen_height / frame_height * y

                    # Thumb (landmark 4)
                    if id == 4:
                        cv2.circle(frame, (x, y), 10, (0, 255, 255), -1)
                        thumb_x = screen_width / frame_width * x
                        thumb_y = screen_height / frame_height * y

                # Check distance between index finger and thumb
                if abs(index_y - thumb_y) < 20:
                    pyautogui.click()
                    pyautogui.sleep(1)
                elif abs(index_y - thumb_y) < 100:
                    pyautogui.moveTo(index_x, index_y)

        # Display the frame
        cv2.imshow('Virtual Mouse', frame)

        # Exit on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    # Release resources
    cap.release()
    cv2.destroyAllWindows()
